In [2]:
import numpy as np
import json

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


In [3]:
#Upload and save VGG16
vgg16 = VGG16(weights = 'imagenet')
vgg16.save('vgg16.h5')

W0714 13:40:55.097248 15072 deprecation_wrapper.py:119] From c:\users\nickm\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0714 13:40:55.111248 15072 deprecation_wrapper.py:119] From c:\users\nickm\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 13:40:55.113248 15072 deprecation_wrapper.py:119] From c:\users\nickm\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0714 13:40:55.132248 15072 deprecation_wrapper.py:119] From c:\users\nickm\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. P

In [7]:
def prepare_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [8]:
y = prepare_image("C:\\Users\\nickm\\Datasets\\car-damage-dataset\\car1.jpg")

In [9]:
preds = vgg16.predict(y)

In [10]:
preds.shape

(1, 1000)

In [11]:
CLASS_INDEX = None

In [12]:
CLASS_INDEX_PATH = ("C:\\Users\\nickm\\PythonProjects\\FullStackDataScience\\imagenet_class_index.json")

In [13]:
def get_predictions(preds, top=5):
    global CLASS_INDEX
    
    #load the json file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [14]:
print(get_predictions(preds, top=5))

[[('n03770679', 'minivan', 0.23744562), ('n03930630', 'pickup', 0.10247658), ('n03594945', 'jeep', 0.09047775), ('n04461696', 'tow_truck', 0.077993274), ('n03345487', 'fire_engine', 0.06803713)]]


In [40]:
from collections import Counter, defaultdict
import os
import pickle as pk

In [36]:
def get_car_categories():
    d = defaultdict(float)
    img_list = os.listdir("C:\\Users\\nickm\\Datasets\\car-damage-dataset\\data1a\\training\\01-whole")
    for i, img_path in enumerate(img_list):
        img = prepare_image("C:\\Users\\nickm\\Datasets\\car-damage-dataset\\data1a\\training\\01-whole\\"+img_path)
        out = vgg16.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:
            d[j[0:2]] += j[2]
        if i % 100 == 0:
                print(i, '/', len(img_list), 'complete')
    return Counter(d)

In [37]:
cat_counter=get_car_categories()

0 / 920 complete
100 / 920 complete
200 / 920 complete
300 / 920 complete
400 / 920 complete
500 / 920 complete
600 / 920 complete
700 / 920 complete
800 / 920 complete
900 / 920 complete


In [38]:
cat_counter

Counter({('n04252225', 'snowplow'): 7.214712330314796,
         ('n03478589', 'half_track'): 0.9229720146395266,
         ('n03126707', 'crane'): 0.12060468643903732,
         ('n04461696', 'tow_truck'): 18.459159269928932,
         ('n03417042', 'garbage_truck'): 1.4169612447731197,
         ('n02701002', 'ambulance'): 5.364399856189266,
         ('n03977966', 'police_van'): 10.064514737401623,
         ('n03345487', 'fire_engine'): 2.661024481523782,
         ('n03670208', 'limousine'): 53.98529281327501,
         ('n03594945', 'jeep'): 29.954810619237833,
         ('n03930630', 'pickup'): 64.39177196170203,
         ('n04467665', 'trailer_truck'): 3.2582269469858147,
         ('n02814533', 'beach_wagon'): 67.72654119720391,
         ('n03770679', 'minivan'): 131.2259159089299,
         ('n02974003', 'car_wheel'): 17.08563916446292,
         ('n03100240', 'convertible'): 91.87715667905286,
         ('n03459775', 'grille'): 17.94024488402647,
         ('n04285008', 'sports_car'): 116.

In [41]:
with open('cat_counter.pk', 'wb') as f:
    pk.dump(cat_counter,f,-1)

In [43]:
cat_list = [k for k, v in cat_counter.most_common()[:27]]

In [44]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03345487', 'fire_engine'),
 ('n03445924', 'golfcart'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

# Car Checker

In [49]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    print("Validating the picture of your car...")
    for j in top[0]:
        if j[0:2] in cat_list:
            print(j[0:2])
            return "Validation complete - proceeding to damage evaluation."
    return "Please try another picture of your car from a different angle or with different lighting."

In [46]:
car_categories_gate("C:\\Users\\nickm\\Datasets\\car-damage-dataset\\car1.jpg", cat_list)

Validating that this is a picture of your car...
('n03770679', 'minivan')


'Validation complete - proceeding to damage evaluation.'